In [1]:
from dotenv import load_dotenv
import os
import base64
import requests
import pandas as pd
import numpy as np

In [2]:
# Get configuration file that has all the api tokens
def configure():
    load_dotenv()

In [3]:
#Grab the Client_id and Client_secret from the env for the documented api
configure()
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
print(CLIENT_ID)
print(CLIENT_SECRET)

40692bc74b7f42f9a0d19365e7af4cd8
790e90e05ada42339bc081c69ffef37e


***

In [4]:
def get_token():
    '''Takes the Client_ID and Client_Secret to request access token.
        Access tokens expire every hour so have to request new one'''
    ## Setup the authorization str and convert to base64  
    auth_str = CLIENT_ID + ":" + CLIENT_SECRET
    #encode str with utf-8 first
    auth_bytes = auth_str.encode("utf-8")
    #encoding it to required base64
    auth_base64 = str(base64.b64encode(auth_bytes),"utf-8")

    ## Run the Actual Request with Post, setting up required header fields
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }

    data = {"grant_type": "client_credentials"}

    ## actually make the request to Spotify
    result = requests.post(url, headers = headers, data = data)
    json_result = result.json()
    token = json_result["access_token"]
    return token

In [5]:
def get_auth_header(token):
    '''Creates Authorization header with the access token for requests to official Spotify API'''
    header = {
        "Authorization": "Bearer " + token
        }
    return header

### Search for artist_id by artist_name

In [6]:
def search_artist(token, artist_name):
    '''Search the Artist name and get their info including ID and popularity'''
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)

    querystring = {
                "q": artist_name,
                "type": "artist",
                "limit": 1
                }

    result = requests.request("GET",url, headers= headers, params=querystring)
    
    json_result = result.json()

    try:
        #returns a artist_id, artist name,
        answer = json_result['artists']['items'][0]
        artist_info = {'artist_name':answer['name'],'artist_id':answer['id'], 'artist_followers':answer['followers']['total'], 'artist_popularity':answer['popularity'], 'artist_genres':answer['genres']}
        return artist_info
    except:
        #give error
        print("No artist with this name on Spotify")
        return None
    

Test what `search_artist()` return

In [7]:
token = get_token()
artist = search_artist(token, "Martin Garrix")

In [8]:
artist

{'artist_name': 'Martin Garrix',
 'artist_id': '60d24wfXkVzDSfLS6hyCjZ',
 'artist_followers': 15731182,
 'artist_popularity': 80,
 'artist_genres': ['dance pop',
  'dutch edm',
  'edm',
  'electro house',
  'pop',
  'pop dance',
  'progressive house',
  'tropical house']}

In [9]:
artist['artist_id']

'60d24wfXkVzDSfLS6hyCjZ'

In [10]:
token

'BQACKvi74KC0NAPRWmcwSwPyEtWeo-e4EPr71CxBtAHSOii1sAIlN8RWyNzaynzzuXvFGdNzGEev2HhPOjx5fGXiS24qBpMoZihDtu2kasL-pi4jiJ4u'

 
### Search for albums and singles by artist_id

In [11]:
def get_singles_albums_by_artist_limit_50(token, artist_id,offset):
    '''Search "album" and "singles" given artist_id limit 50 per request at a time
        Return list of items '''
    url = f"	https://api.spotify.com/v1/artists/{artist_id}/albums"
    headers = get_auth_header(token)

    querystring = {
                "include_groups": "album,single",
                "limit": 50,
                "offset": offset
                }

    result = requests.request("GET",url, headers= headers, params=querystring)
    
    json_result = result.json()
    items = json_result['items']
    return items

Test what `get_singles_albums_by_artist_limit_50` return

In [12]:
#testing the first 50
album_single = get_singles_albums_by_artist_limit_50(token,artist['artist_id'],0)
album_single

[{'album_group': 'album',
  'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/60d24wfXkVzDSfLS6hyCjZ'},
    'href': 'https://api.spotify.com/v1/artists/60d24wfXkVzDSfLS6hyCjZ',
    'id': '60d24wfXkVzDSfLS6hyCjZ',
    'name': 'Martin Garrix',
    'type': 'artist',
    'uri': 'spotify:artist:60d24wfXkVzDSfLS6hyCjZ'}],
  'available_markets': ['AR',
   'AU',
   'AT',
   'BE',
   'BO',
   'BR',
   'BG',
   'CA',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DK',
   'DO',
   'DE',
   'EC',
   'EE',
   'SV',
   'FI',
   'FR',
   'GR',
   'GT',
   'HN',
   'HU',
   'IS',
   'IE',
   'IT',
   'LV',
   'LT',
   'LU',
   'MY',
   'MT',
   'MX',
   'NL',
   'NZ',
   'NI',
   'NO',
   'PA',
   'PY',
   'PE',
   'PH',
   'PL',
   'PT',
   'SG',
   'SK',
   'ES',
   'SE',
   'CH',
   'TR',
   'UY',
   'US',
   'GB',
   'AD',
   'LI',
   'MC',
   'ID',
   'JP',
   'TH',
   'VN',
   'RO',
   'IL',
   'ZA',
   'SA',
   'AE',
   'BH',
   'QA',
   'OM',

In [13]:
def get_all_singles_albums_by_artist(token, artist_id):
    '''Grabs all the singles and albums for the artist and return a dataframe
        with album_group, name, id, release_date, total_tracks, uri'''
    offset_counter = 0
    data = []
    while True:
        results = get_singles_albums_by_artist_limit_50(token,artist_id,offset_counter)
        offset_counter += 50
        data.extend(results)

        if len(results) != 50:
            break

    dataframe = pd.DataFrame(data, columns = ["album_group","name","id","release_date","total_tracks",'uri'])
    return dataframe


In [14]:
all_albums = get_all_singles_albums_by_artist(token,artist['artist_id'])
all_albums

,album_group,name,id,release_date,total_tracks,uri
0,album,Sentio,45gsxfnJ5Nt2RZp82SQenc,2022-04-29,11,spotify:album:45gsxfnJ5Nt2RZp82SQenc
1,album,Sentio,3hmkWvkDpa5Og7OFvtsy5x,2022-04-29,11,spotify:album:3hmkWvkDpa5Og7OFvtsy5x
2,album,Greatest Hits Vol. 1,30WNa86MJsrzTlki1YHI6A,2021-11-12,12,spotify:album:30WNa86MJsrzTlki1YHI6A
3,album,The Martin Garrix Experience,3UwcZ5menPSCTvdxyYgxZj,2019-07-10,20,spotify:album:3UwcZ5menPSCTvdxyYgxZj
4,single,Cocoon (Martin Garrix & Space Ducks Remix),11X8fP4LGagH6xsQI2aqaY,2023-02-10,1,spotify:album:11X8fP4LGagH6xsQI2aqaY
...,...,...,...,...,...,...
269,single,Keygen,0oM1EtVZPU9epXTG12NYDO,2012-09-03,1,spotify:album:0oM1EtVZPU9epXTG12NYDO
270,single,Keygen,3QFvAEkxIgVcexCdJlS9nk,2012-09-03,1,spotify:album:3QFvAEkxIgVcexCdJlS9nk
271,single,ITSA,512dWmPruPaity5oO8yYff,2012-04-09,1,spotify:album:512dWmPruPaity5oO8yYff
272,single,BFAM,6pNLWCEMQCREaJQYVq6saF,2012,1,spotify:album:6pNLWCEMQCREaJQYVq6saF
